In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from segmentation.loader import load
from segmentation.model import ResnetCrackModel
from torchvision import transforms
import torch

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

data = load("3", transform=transform)

In [4]:
model = ResnetCrackModel()
model.train(data.train_loader, data.val_loader)

/home/warlock/repos/segmentation/.venv/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/10: Training:  26%|███████████████████████████████████████▉                                                                                                                 | 55/211 [00:40<01:55,  1.36it/s]


KeyboardInterrupt: 

In [6]:
model

device(type='cuda')